In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
###### Main Module
#####
###########################
###### Packages
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from tabulate import tabulate

import time
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

# Algos
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

###########################

In [4]:
labels_file=pd.read_csv("/content/drive/MyDrive/IntrusionDetection/CIC_IOT_2023/02-TreatedDataset/CICIoT2023-34_labels.csv",header=None)

In [5]:
labels_file.value_counts()

DDoS-ICMP_Flood            360668
DDoS-UDP_Flood             270507
DDoS-TCP_Flood             224345
DDoS-PSHACK_Flood          203975
DDoS-SYN_Flood             203438
DDoS-RSTFINFlood           202181
DDoS-SynonymousIP_Flood    180243
DoS-UDP_Flood              165922
DoS-TCP_Flood              133678
DoS-SYN_Flood              101620
BenignTraffic               54926
Mirai-greeth_flood          49964
Mirai-udpplain              44110
Mirai-greip_flood           37642
DDoS-ICMP_Fragmentation     22488
MITM-ArpSpoofing            15530
DDoS-UDP_Fragmentation      14454
DDoS-ACK_Fragmentation      14183
DNS_Spoofing                 8884
Recon-HostDiscovery          6664
Recon-OSScan                 4850
Recon-PortScan               3999
DoS-HTTP_Flood               3583
VulnerabilityScan            1924
DDoS-HTTP_Flood              1414
DDoS-SlowLoris               1127
DictionaryBruteForce          635
CommandInjection              290
BrowserHijacking              288
SqlInjection  

In [15]:
data1 = np.load('/content/drive/MyDrive/IntrusionDetection/CIC_IOT_2023/03-Experiments/2024-06-26/CICIoT2023-MS-s100.npy', allow_pickle=True).item()
data2 = np.load('/content/drive/MyDrive/IntrusionDetection/CIC_IOT_2023/03-Experiments/2024-06-26/CICIoT2023-MS-s200.npy', allow_pickle=True).item()
data3 = np.load('/content/drive/MyDrive/IntrusionDetection/CIC_IOT_2023/03-Experiments/2024-06-26/CICIoT2023-MS-s300.npy', allow_pickle=True).item()
data4 = np.load('/content/drive/MyDrive/IntrusionDetection/CIC_IOT_2023/03-Experiments/2024-06-26/CICIoT2023-MS-s400.npy', allow_pickle=True).item()
data5 = np.load('/content/drive/MyDrive/IntrusionDetection/CIC_IOT_2023/03-Experiments/2024-06-26/CICIoT2023-MS-s500.npy', allow_pickle=True).item()

In [10]:
data1[80068]

{'querying_time': [7.025927699985914],
 'classification_time': [6.612062200001674],
 'accuracy': [0.9904697578313202],
 'balanced_accuracy': [0.7394509543115587],
 'selected_indices': [[620957,
   1443528,
   598499,
   1068827,
   606233,
   212320,
   523522,
   757449,
   797740,
   1083829,
   272237,
   1347194,
   970939,
   600327,
   837540,
   139483,
   1485634,
   1360126,
   482818,
   1132622,
   294394,
   63006,
   492112,
   626996,
   89295,
   83524,
   509845,
   885677,
   263094,
   1535837,
   1239358,
   729084,
   589012,
   222147,
   1057837,
   1280490,
   1501561,
   420714,
   1127437,
   972310,
   581506,
   1043322,
   1280486,
   47335,
   1207766,
   748349,
   93337,
   1458998,
   266406,
   1337875,
   849555,
   1091387,
   806067,
   451976,
   1136580,
   250317,
   179836,
   1020818,
   886702,
   1025558,
   329170,
   708423,
   1337836,
   734266,
   1015925,
   1469124,
   841259,
   418430,
   1509248,
   346521,
   355351,
   243226,
   3

In [11]:
querying_time1 = []
classification_time1 = []
for key, value in data1.items():
    if isinstance(key, int) and 'balanced_accuracy' in value:
        querying_time1.append(value['querying_time'][0])
        classification_time1.append(value['classification_time'][0])

print(len(classification_time1[0:161]))
print((classification_time1[0:161]))

# 6.612062200001674

161
[0.022423799993703142, 0.028271200018934906, 0.08103309999569319, 0.11633469999651425, 0.09556029998930171, 0.1961684999987483, 0.25026339999749325, 0.20421799999894574, 0.30105249999905936, 0.51833120000083, 0.3469021999917459, 0.4552489999914542, 0.5168734000180848, 0.4689304000057746, 0.4571965000068303, 0.7001556000031997, 0.8067494000133593, 0.7691535000049043, 0.7701443999831099, 0.8552249000058509, 0.9254978999961168, 1.0168670999992173, 0.9694501000049058, 1.256326100003207, 1.1255106999888085, 1.075716499995906, 1.1767480999988038, 1.3699421999917831, 1.278903700003866, 1.1075454000092577, 1.5369101999967825, 1.4966506000200752, 1.491539600014221, 1.5578218000009656, 1.7849148999957833, 1.6378114000253845, 1.5323563999845646, 2.0157477000029758, 1.7213172999909148, 1.8371436000161339, 1.4565868000208866, 1.8865802999935113, 2.1819032999919727, 1.9462664000166114, 2.1130857999960426, 2.1160131000215188, 2.15551989999949, 2.2096480000182055, 2.4468712999951094, 2.38435330000

In [17]:
querying_time1 = []
querying_time2 = []
querying_time3 = []
querying_time4 = []
querying_time5 = []
classification_time1 = []
classification_time2 = []
classification_time3 = []
classification_time4 = []
classification_time5 = []

for key, value in data1.items():
    if isinstance(key, int) and 'balanced_accuracy' in value:
        querying_time1.append(value['querying_time'][0])
        classification_time1.append(value['classification_time'][0])

for key, value in data2.items():
    if isinstance(key, int) and 'balanced_accuracy' in value:
        querying_time2.append(value['querying_time'][0])
        classification_time2.append(value['classification_time'][0])

for key, value in data3.items():
    if isinstance(key, int) and 'balanced_accuracy' in value:
        querying_time3.append(value['querying_time'][0])
        classification_time3.append(value['classification_time'][0])

for key, value in data4.items():
    if isinstance(key, int) and 'balanced_accuracy' in value:
        querying_time4.append(value['querying_time'][0])
        classification_time4.append(value['classification_time'][0])

for key, value in data5.items():
    if isinstance(key, int) and 'balanced_accuracy' in value:
        querying_time5.append(value['querying_time'][0])
        classification_time5.append(value['classification_time'][0])

print("querying_time médio:", (sum(querying_time1[0:161]) + \
                               sum(querying_time2[0:161]) + \
                               sum(querying_time3[0:161]) + \
                               sum(querying_time4[0:161]) + \
                               sum(querying_time5[0:161]) )/5/60)

print("querying_time desvio:" , np.std(np.array([sum(querying_time1[0:161])/60 , \
                                    sum(querying_time2[0:161])/60 , \
                                    sum(querying_time3[0:161])/60 , \
                                    sum(querying_time4[0:161])/60 , \
                                    sum(querying_time5[0:161])/60 ])))

print("query time iteration média:",(  data1[80068]['querying_time'][0] + \
                                    data2[80068]['querying_time'][0] + \
                                    data3[80068]['querying_time'][0] + \
                                    data4[80068]['querying_time'][0] + \
                                    data5[80068]['querying_time'][0] )/5)

print("query time iteration desvio:" , np.std(np.array([ data1[80068]['querying_time'][0] , \
                                    data2[80068]['querying_time'][0] , \
                                    data3[80068]['querying_time'][0] , \
                                    data4[80068]['querying_time'][0] , \
                                    data5[80068]['querying_time'][0] ])))

print("classification_time médio:",( sum(classification_time1[0:161]) + \
                                    sum(classification_time2[0:161]) + \
                                    sum(classification_time3[0:161]) + \
                                    sum(classification_time4[0:161]) + \
                                    sum(classification_time5[0:161]) )/5/60)

print("classification_time desvio:", np.std(np.array([sum(classification_time1[0:161])/60 , \
                                    sum(classification_time2[0:161])/60 , \
                                    sum(classification_time3[0:161])/60 , \
                                    sum(classification_time4[0:161])/60 , \
                                    sum(classification_time5[0:161])/60 ])))

print("classification_time iteration média:",(  data1[80068]['classification_time'][0] + \
                                    data2[80068]['classification_time'][0] + \
                                    data3[80068]['classification_time'][0] + \
                                    data4[80068]['classification_time'][0] + \
                                    data5[80068]['classification_time'][0] )/5)

print("classification_time iteration desvio:" , np.std(np.array([ data1[80068]['classification_time'][0] , \
                                    data2[80068]['classification_time'][0] , \
                                    data3[80068]['classification_time'][0] , \
                                    data4[80068]['classification_time'][0] , \
                                    data5[80068]['classification_time'][0] ])))

print("Classes média:",(            data1[80068]['classes_know'][0] + \
                                    data2[80068]['classes_know'][0] + \
                                    data3[80068]['classes_know'][0] + \
                                    data4[80068]['classes_know'][0] + \
                                    data5[80068]['classes_know'][0] )/5)

print("Classes desvio:" , np.std(np.array([ data1[80068]['classes_know'][0] , \
                                    data2[80068]['classes_know'][0] , \
                                    data3[80068]['classes_know'][0] , \
                                    data4[80068]['classes_know'][0] , \
                                    data5[80068]['classes_know'][0] ])))

print("Acurácia média:",(           data1[80068]['accuracy'][0] + \
                                    data2[80068]['accuracy'][0] + \
                                    data3[80068]['accuracy'][0] + \
                                    data4[80068]['accuracy'][0] + \
                                    data5[80068]['accuracy'][0] )/5)

print("Acurácia desvio:" , np.std(np.array([ data1[80068]['accuracy'][0] , \
                                    data2[80068]['accuracy'][0] , \
                                    data3[80068]['accuracy'][0] , \
                                    data4[80068]['accuracy'][0] , \
                                    data5[80068]['accuracy'][0] ])))

print("balanced accuracy média:",(  data1[80068]['balanced_accuracy'][0] + \
                                    data2[80068]['balanced_accuracy'][0] + \
                                    data3[80068]['balanced_accuracy'][0] + \
                                    data4[80068]['balanced_accuracy'][0] + \
                                    data5[80068]['balanced_accuracy'][0] )/5)

print("balanced accuracy desvio:" , np.std(np.array([ data1[80068]['balanced_accuracy'][0] , \
                                    data2[80068]['balanced_accuracy'][0] , \
                                    data3[80068]['balanced_accuracy'][0] , \
                                    data4[80068]['balanced_accuracy'][0] , \
                                    data5[80068]['balanced_accuracy'][0] ])))

# Definir as classes únicas esperadas
unique_classes = ['DDoS-ICMP_Flood','DDoS-UDP_Flood','DDoS-TCP_Flood','DDoS-PSHACK_Flood','DDoS-SYN_Flood','DDoS-RSTFINFlood', \
                  'DDoS-SynonymousIP_Flood','DoS-UDP_Flood','DoS-TCP_Flood','DoS-SYN_Flood','BenignTraffic','Mirai-greeth_flood', \
                  'Mirai-udpplain','Mirai-greip_flood','DDoS-ICMP_Fragmentation','MITM-ArpSpoofing','DDoS-UDP_Fragmentation', \
                  'DDoS-ACK_Fragmentation','DNS_Spoofing','Recon-HostDiscovery','Recon-OSScan','Recon-PortScan','DoS-HTTP_Flood', \
                  'VulnerabilityScan','DDoS-HTTP_Flood','DDoS-SlowLoris','DictionaryBruteForce','CommandInjection','BrowserHijacking', \
                  'SqlInjection','XSS','Backdoor_Malware','Recon-PingSweep','Uploading_Attack'
                  ]

# Inicializar um dicionário para armazenar as contagens de todas as classes
class_counts_all = {cls: 0 for cls in unique_classes}

# Concatenar todos os vetores em um único array
all_classes = np.concatenate((
    data1[80068]['classes'][0],
    data2[80068]['classes'][0],
    data3[80068]['classes'][0],
    data4[80068]['classes'][0],
    data5[80068]['classes'][0]
))

# Contar as ocorrências de cada classe encontrada
unique_classes_found, class_counts_found = np.unique(all_classes, return_counts=True)

# Atualizar o dicionário com as contagens encontradas
for cls, count in zip(unique_classes_found, class_counts_found):
    if cls in class_counts_all:
        class_counts_all[cls] = count

# Verificar e adicionar zero para classes não encontradas
for cls in unique_classes:
    if cls not in class_counts_all:
        class_counts_all[cls] = 0

# Preparar os dados para a tabela
table_data = []
for cls in unique_classes:
    table_data.append([cls, class_counts_all[cls]])

# Imprimir a tabela usando tabulate
print(tabulate(table_data, headers=['Classe', 'Frequência'], tablefmt='grid'))

querying_time médio: 19.964342864667124
querying_time desvio: 6.738471688823675
query time iteration média: 7.2296542399912145
query time iteration desvio: 2.4945476082628626
classification_time médio: 9.882745994663855
classification_time desvio: 4.431255615076302
classification_time iteration média: 7.305327600007876
classification_time iteration desvio: 3.455916721259895
Classes média: 32.6
Classes desvio: 0.4898979485566356
Acurácia média: 0.9928907306471396
Acurácia desvio: 0.00014511278050953182
balanced accuracy média: 0.7377829323092528
balanced accuracy desvio: 0.004835568407829553
+-------------------------+--------------+
| Classe                  |   Frequência |
+=========================+==============+
| DDoS-ICMP_Flood         |            5 |
+-------------------------+--------------+
| DDoS-UDP_Flood          |            5 |
+-------------------------+--------------+
| DDoS-TCP_Flood          |            5 |
+-------------------------+--------------+
| DDoS-PSHACK_F